# Main file for thesis project

Ie the top level script which runs the whole `pic -> recognise -> aim gimbal -> repeat` show

In [ ]:
import time, numpy as np, matplotlib.pyplot as plt

# homemade stuff
import gimbal_control as gc
from extended_kalman_filter import ExtendedKalmanFilter
from image_classifier import ImageClassifier
from utils import plot_util, ExperimentLogger

# Constants + things which need global scope

In [ ]:
total_run_time = 5    # seconds
t_loop_desired = 0.05 # seconds = 50 ms

# initialize variables in case the neural net doesn't spot anything the first time round
phi_yaw, phi_pitch = 0, 0

## A Basic Loop

In [ ]:
# setup stuff
EKF_yaw   = ExtendedKalmanFilter(Ts=0.05, Q=0.5, R=0.005, a=0.94, camera_FOV_deg=62.2)  # need to tweak Q and R
EKF_pitch = ExtendedKalmanFilter(Ts=0.05, Q=0.5, R=0.005, a=0.94, camera_FOV_deg=48.8)  # need to tweak Q and R

IC = ImageClassifier(
    graph_filename='../Models/MobileNet_SSD_caffe/graph',
    label_filename='../Models/MobileNet_SSD_caffe/categories.txt',
    class_of_interest='person',
    camera_resolution=(1640, 1232),#(1640, 922)
    camera_FOV_deg=(62.2, 48.8),
    photo_logging_params=(10000, 'logged_photos/'),
    debug=True)

# logging used for offline stuff only
EL = ExperimentLogger()

# a rolling buffer for the gimbal angles
gimbal_buffer = GimbalAngleBuffer()

In [ ]:
while IC.dict_queue.empty() is True: pass # wait until there's a result from the nn (even if it didn't spot an object)

t_start = time.time()
t_since_nn_update = time.time()

EKF_yaw.start_timer()
EKF_pitch.start_timer()

while True:
    t = time.time()
    
    ######################### LOG THE GIMBAL ANGLES #########################
    gc_angles = gc.get_motor_angles()
    gc_angles['yaw'] = ((gc_angles['yaw']+90) % 180)-90 # get rid of erroneous spikes
    gc_angles['pitch'] = ((-gc_angles['pitch']+90) % 180)-90
    gimbal_buffer.log(gc_angles)
    
    ######################### NN and EKF #########################
    # if there is a new result waiting...
    if not IC.dict_queue.empty():
        bb, bb_angles, photo_time = IC.get_result()

        # and there was an actual object detected...
        if bb_angles != -1:
            (phi_x1, phi_y1), (phi_x2, phi_y2) = bb_angles
            phi_yaw = (phi_x1 + phi_x2)/2
            phi_pitch = (phi_y1 + phi_y2)/2 # come as negative angles
            
#             print('photo -> result =', (time.time()-photo_time)*1000)
            logger_yaw, logger_pitch = gimbal_buffer.closest_angle_to(photo_time)            
            EKF_yaw.better_update(phi_yaw + logger_yaw)
            EKF_pitch.better_update(phi_pitch + logger_pitch)
            
            t_since_nn_update = time.time()# TODO: this might not be the best approach! maybe do photo_time??
#             t_since_nn_update = photo_time
    
    ######################### CONTROL #########################
    desired_yaw = EKF_yaw.predict_ahead(time.time() - t_since_nn_update)
    desired_pitch = EKF_pitch.predict_ahead(time.time() - t_since_nn_update)
    
    # add in bounds
    desired_pitch = min(max(desired_pitch, -45), 90) # bounds are (-45, 90)
    desired_yaw = min(max(desired_yaw, -60), 60)
    
    gc.send_angle_command(roll=0, pitch=20, yaw=desired_yaw)

    ######################### TIMING #########################    
    t_end = time.time()
    if t_end > t_start + total_run_time:
        break
    elif t_end - t > t_loop_desired:
        EL.log(gc_angles, phi_yaw, EKF_yaw, phi_pitch, EKF_pitch, time.time() - t_start)
    else:
        time.sleep(t_loop_desired - (t_end - t))  # aim for a loop time of 50ms
        EL.log(gc_angles, phi_yaw, EKF_yaw, phi_pitch, EKF_pitch, time.time() - t_start)

IC.close()
gc.send_angle_command(0, 0, 0);

In [ ]:
IC.close()
gc.send_angle_command(0, 0, 0);